In [1]:
import os
import glob
import pandas as pd
import tensorflow as tf
import numpy as np

from IPython.display import display, HTML

In [1]:
number_of_crossvalidation_run=10
cwd=os.getcwd()
visual_m_path = os.path.join(cwd,'Desktop/book-cover/imgs_mean_feature_vectors.pkl')
textual_m_path = os.path.join(cwd,'Desktop/book-cover/docs_extracted_features.pkl')
models_folder_name = os.path.join(cwd,'models',str(number_of_crossvalidation_run))
path_to_save_test_results=os.path.join(models_folder_name, 'test_results.pkl')
model_checkpoint_path = os.path.join(cwd,models_folder_name,'gmu.ckpt')
models_folder_name_test = os.path.join(cwd,os.pardir,'Desktop/book-cover/text_feature_extraction','models','test',str(number_of_crossvalidation_run))
path_to_preprocessed_texts_test=os.path.join(models_folder_name_test,'book-cover_test_dataset.pkl')
summaries_folder_name=os.path.join(cwd,'summaries',str(number_of_crossvalidation_run))


df_visual_m = pd.read_pickle(visual_m_path)
df_textual_m = pd.read_pickle(textual_m_path)

number_of_recipes=len(df_visual_m)
unique_labels=sorted(set(df_visual_m.mean_vector_labels.values))
number_of_classes=len(unique_labels)
possible_class_indices=list(range(0,number_of_classes))
labels2class_indices=dict(zip(unique_labels,possible_class_indices))
print(labels2class_indices)

Test preparation" : "0" "Calendar":"1" "Comics, graphics & novels" :"2" "Romance" :"3""Computer" "&" "technology ":"4"")
"Cookbooks, food & wine":5"Children books" :6"Science fiction and fantasy":7
"Mystery, thriller & suspense ":8"Travel ":9"Law" :10"Reference" :11
"Parenting and relationship ":12"Engineering & transportation ":13"Medical books ":14
"Crafts ,hobbies & home ":15"Religion and spirituality ":16"Sports and outdoors ":18"Self  help ":19
"Science & math ":20"Biography & memoirs" :21"Business & money ":22
"Health fitness & dieting ":23"History ":24"Teen & young Adults ":2 "Arts & photography":26"Literature & Fiction":27
"Humor & Entertainment ":28"Christian books & Bible ":29 "Political & social sciences":30


In [ ]:
not_normalized_features_visual=df_visual_m.mean_feature_vectors.tolist()
mean_visual=np.mean(not_normalized_features_visual)
#print("Mean value: visual features ", mean_visual)
subtracted_mean_visual= (not_normalized_features_visual-mean_visual)
max_visual=np.max(subtracted_mean_visual)
print("Max value in visual feature vectors after subraction of mean value: ", max_visual)
normalized_features_visual = (subtracted_mean_visual/max_visual).tolist()
df_visual_m.mean_feature_vectors=normalized_features_visual

not_normalized_features_textual=df_textual_m.doc_embeddings.tolist()
mean_textual=np.mean(not_normalized_features_textual)
#print("Mean value: textual features ", mean_textual)
subtracted_mean_textual= (not_normalized_features_textual-mean_textual)
max_textual=np.max(subtracted_mean_textual)
#print("Max value in textual feature vectors after subraction of mean value: ", max_textual)
normalized_features_textual = (subtracted_mean_textual/max_textual).tolist()
df_textual_m.doc_embeddings=normalized_features_textual


df_textual_m


In [4]:
df_preprocessed_texts_test = pd.read_pickle(path_to_preprocessed_texts_test)
test_samples=df_preprocessed_texts_test.index.values.tolist()

all_samples=set(range(0,number_of_recipes))
train_samples=list(all_samples.difference(test_samples))
print("train samples: ", train_samples)
print("test samples: ", test_samples)

train samples:  [1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 40, 41, 42, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 97, 98, 100, 101, 102, 103, 104, 106, 107, 108, 109, 110, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 126, 127, 128, 129, 130, 131, 132, 134, 135, 136, 137, 139, 140, 141, 142, 143, 144, 145, 146, 148, 149, 150, 151, 153, 154, 155, 156, 158, 159, 160, 161, 162, 164, 165, 167, 168, 169, 170, 171, 173, 174, 175, 176, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 191, 192, 193, 194, 195, 196, 198, 199, 200, 201, 203, 204, 205, 206, 207, 208, 209, 211, 212, 213, 214, 215, 216, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 230, 231, 232, 234, 235, 236, 237, 238, 240, 241, 243, 245, 247, 248, 249, 250, 251, 252, 253, 254, 255,

In [5]:
df_visual_m_train=df_visual_m.iloc[train_samples]
df_visual_m_test=df_visual_m.iloc[test_samples]
df_textual_m_train=df_textual_m.iloc[train_samples]
df_textual_m_test=df_textual_m.iloc[test_samples]

visual_m_train_inputs=list(df_visual_m_train.mean_feature_vectors.values)
visual_m_test_inputs=list(df_visual_m_test.mean_feature_vectors.values)

textual_m_train_inputs=list(df_textual_m_train.doc_embeddings.values)
textual_m_test_inputs=list(df_textual_m_test.doc_embeddings.values)

train_correct_class_ids=[labels2class_indices[l] for l in df_visual_m_train.mean_vector_labels]
test_correct_class_ids=[labels2class_indices[l] for l in df_visual_m_test.mean_vector_labels]

number_of_training_samples=len(visual_m_train_inputs)
number_of_test_samples=len(visual_m_test_inputs)
len_of_visual_features_vec=len(visual_m_train_inputs[0])
len_of_textual_features_vec=len(textual_m_train_inputs[0])

print(test_correct_class_ids)
print(np.shape(visual_m_train_inputs))
print(np.shape(textual_m_train_inputs))

[0, 0, 0, 1, 1, 2, 3, 3, 3, 4, 4, 5, 6, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 13, 13, 14, 14, 15, 15, 15, 16, 17, 17, 18, 19, 20, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23, 24, 24, 25, 25, 26, 26]
(264, 202)
(264, 27)


In [6]:
batch_size=25
learning_rate=0.001
hidden_state_dim = 27
epochs=420
print_valid_every=20
num_repeat_training=10

In [ ]:
df_textual_m_test

In [8]:
def remove_directory_content(path):
    files = glob.glob(path+"/*")
    for f in files:
        os.remove(f)

def create_training_batch():
    inputs_visual=[]
    inputs_textual=[]
    correct_classes=[]
    for i in range(batch_size):
        train_sample_index=np.random.choice(range(0,number_of_training_samples),1)[0]
        inputs_visual.append(visual_m_train_inputs[train_sample_index])
        inputs_textual.append(textual_m_train_inputs[train_sample_index])
        correct_classes.append(train_correct_class_ids[train_sample_index])
    return np.array(inputs_visual),np.array(inputs_textual),np.array(correct_classes)

inputs_visual,inputs_textual,correct_classes=create_training_batch()
print(np.array(correct_classes))
print(np.shape(inputs_visual))
print(np.shape(inputs_textual))
print(np.shape(correct_classes))

[ 3  8 22  9]
(4, 202)
(4, 27)
(4,)


In [9]:
visual = tf.placeholder(tf.float32, shape=[None,len_of_visual_features_vec],name="input_visual_modality")
textual = tf.placeholder(tf.float32, shape=[None,len_of_textual_features_vec], name="input_textual_modality")
target = tf.placeholder(tf.int32, shape=[None],name="input_correct_labels")

h_v = tf.layers.dense(visual, hidden_state_dim, activation=tf.nn.tanh, name="h_v")

h_t = tf.layers.dense(textual, hidden_state_dim, activation=tf.nn.tanh, name="h_t")

z = tf.layers.dense(tf.concat([visual,textual], axis=1),
                    hidden_state_dim,
                    kernel_initializer=tf.initializers.random_uniform,
                    activation=tf.nn.sigmoid,
                    name="z")

h = tf.concat([(1-z) * h_v, z * h_t], axis=1, name="h_concat")


logits = tf.layers.dense(h, number_of_classes, name="h")
scores = tf.nn.sigmoid(logits)

multi_class_labels=tf.one_hot(target, depth=number_of_classes)
loss = tf.reduce_mean(tf.losses.sigmoid_cross_entropy(multi_class_labels=multi_class_labels,
                                       logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

correct_prediction=tf.equal(tf.argmax(logits, axis=1), tf.argmax(multi_class_labels,axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.name_scope("performance"):
    loss_ph = tf.placeholder(tf.float32,shape=None,name='loss_summary')
    loss_summary = tf.summary.scalar('loss', loss_ph)
    accuracy_ph = tf.placeholder(tf.float32,shape=None,name='accuracy_summary')
    accuracy_summary = tf.summary.scalar('accuracy', accuracy_ph)

saver = tf.train.Saver()

In [10]:
best_accuracy=0

def show_validation_result(session,accuracy_res, loss_res, scores_res,z_res):
    print("Validation: Loss: ", loss_res," Accuracy: ", accuracy_res)
    global best_accuracy
    if accuracy_res > best_accuracy:
        best_accuracy=accuracy_res
        df_performance=pd.DataFrame(data={'Accuracy':[accuracy_res], 'Loss':[loss_res]})
        class_scores=list(np.around(scores_res,4))
        predicted_labels = [np.argmax(one_recipe_class_scores) for one_recipe_class_scores in class_scores]
        df_scores=pd.DataFrame(data={'Class scores':class_scores,
                               'Predicted labels':predicted_labels,
                               'Correct labels':test_correct_class_ids,
                               'Trust to visual modality': list(np.around(1-z_res,4)),
                               'Trust to textual modality': list(np.around(z_res,4))})
        df_res=pd.concat([df_scores, df_performance], axis=1)
        df_res.to_pickle(path_to_save_test_results)
        saver.save(session, model_checkpoint_path)
        display(df_res)
        
    

def train(num_of_run):
        session=tf.InteractiveSession()
        summ_writer = tf.summary.FileWriter(summaries_folder_name, session.graph)
        session.run(tf.global_variables_initializer())
    
        print("Start model training")
    
        for train_iter in range(number_of_training_iterations):           
            inputs_visual,inputs_textual,correct_classes=create_training_batch()
            _, l = session.run([train_op, loss], {visual: inputs_visual,
                                           textual: inputs_textual,
                                           target: correct_classes})
            print(train_iter, ": Training: loss: ", l)
            summ_loss = session.run(loss_summary, feed_dict={loss_ph:l})
            summ_writer.add_summary(summ_loss, train_iter)
        
            if (train_iter)%print_valid_every==0:
                accuracy_res, loss_res, scores_res,z_res = session.run([accuracy, loss, scores, z],
                                                                          {visual: visual_m_test_inputs,
                                                                           textual: textual_m_test_inputs,
                                                                           target: test_correct_class_ids})        
                
                show_validation_result(session,accuracy_res, loss_res, scores_res,z_res)
                summ_accuracy = session.run(accuracy_summary, feed_dict={accuracy_ph:accuracy_res})
                summ_writer.add_summary(summ_accuracy, train_iter)
                
                if accuracy_res==1.0:
                    return 0
                
        session.close()
        return -1

In [11]:
for num_of_run in range(num_repeat_training):
    remove_directory_content(summaries_folder_name)
    if train(num_of_run)==0:
        break
        

Start model training
0 : Training: loss:  0.68781215
Validation: Loss:  0.6948268  Accuracy:  0.057692308


,Class scores,Correct labels,Predicted labels,Trust to textual modality,Trust to visual modality,Accuracy,Loss
0,"[0.5168, 0.5023, 0.4923, 0.4946, 0.5139, 0.508...",0,0,"[0.1807, 0.2445, 0.1913, 0.1714, 0.2011, 0.142...","[0.8193, 0.7555, 0.8087, 0.8286, 0.7989, 0.857...",0.057692,0.694827
1,"[0.5106, 0.5013, 0.5214, 0.5518, 0.4871, 0.501...",0,3,"[0.4069, 0.5304, 0.4535, 0.3838, 0.5394, 0.480...","[0.5931, 0.4696, 0.5465, 0.6162, 0.4606, 0.519...",NaN,NaN
2,"[0.516, 0.5027, 0.4993, 0.4948, 0.5036, 0.5365...",0,5,"[0.3044, 0.2696, 0.1856, 0.2198, 0.3323, 0.250...","[0.6956, 0.7304, 0.8144, 0.7802, 0.6677, 0.749...",NaN,NaN
3,"[0.4877, 0.4687, 0.4833, 0.5374, 0.5382, 0.494...",1,8,"[0.5863, 0.5355, 0.3545, 0.36, 0.4462, 0.6735,...","[0.4137, 0.4645, 0.6455, 0.64, 0.5538, 0.3265,...",NaN,NaN
4,"[0.4994, 0.4891, 0.5025, 0.5143, 0.5026, 0.509...",1,18,"[0.1891, 0.1245, 0.1281, 0.0603, 0.1483, 0.163...","[0.8109, 0.8755, 0.8719, 0.9397, 0.8517, 0.836...",NaN,NaN
5,"[0.513, 0.4862, 0.4779, 0.5026, 0.518, 0.5102,...",2,25,"[0.1927, 0.1613, 0.1763, 0.4588, 0.2234, 0.309...","[0.8073, 0.8387, 0.8237, 0.5412, 0.7766, 0.690...",NaN,NaN
6,"[0.5867, 0.3823, 0.5024, 0.497, 0.618, 0.514, ...",3,4,"[0.7317, 0.709, 0.6666, 0.7672, 0.6525, 0.715,...","[0.2683, 0.291, 0.3334, 0.2328, 0.3475, 0.285,...",NaN,NaN
7,"[0.5441, 0.4566, 0.4494, 0.4941, 0.551, 0.5175...",3,19,"[0.4488, 0.6199, 0.5263, 0.6516, 0.4803, 0.505...","[0.5512, 0.3801, 0.4737, 0.3484, 0.5197, 0.494...",NaN,NaN
8,"[0.5106, 0.4874, 0.4648, 0.4768, 0.5206, 0.523...",3,19,"[0.368, 0.3414, 0.3981, 0.5211, 0.3463, 0.3776...","[0.632, 0.6586, 0.6019, 0.4789, 0.6537, 0.6224...",NaN,NaN
9,"[0.4606, 0.4876, 0.4985, 0.4546, 0.5113, 0.477...",4,20,"[0.8967, 0.8397, 0.8959, 0.9386, 0.8917, 0.945...","[0.1033, 0.1603, 0.1041, 0.0614, 0.1083, 0.054...",NaN,NaN


1 : Training: loss:  0.69339514
2 : Training: loss:  0.69322824
3 : Training: loss:  0.69540507
4 : Training: loss:  0.69976014
5 : Training: loss:  0.68276405
6 : Training: loss:  0.6863913
7 : Training: loss:  0.6731956
8 : Training: loss:  0.68043834
9 : Training: loss:  0.6817261
10 : Training: loss:  0.69404745
11 : Training: loss:  0.6904331
12 : Training: loss:  0.68174285
13 : Training: loss:  0.67688364
14 : Training: loss:  0.67886806
15 : Training: loss:  0.6783116
16 : Training: loss:  0.6665101
17 : Training: loss:  0.6753495
18 : Training: loss:  0.66955674
19 : Training: loss:  0.6733662
20 : Training: loss:  0.68009984
Validation: Loss:  0.6715678  Accuracy:  0.057692308
21 : Training: loss:  0.66981584
22 : Training: loss:  0.6843944
23 : Training: loss:  0.6677409
24 : Training: loss:  0.6647166
25 : Training: loss:  0.66101265
26 : Training: loss:  0.66064024
27 : Training: loss:  0.6522429
28 : Training: loss:  0.655568
29 : Training: loss:  0.6498703
30 : Training:

,Class scores,Correct labels,Predicted labels,Trust to textual modality,Trust to visual modality,Accuracy,Loss
0,"[0.4939, 0.4753, 0.4399, 0.4716, 0.4768, 0.491...",0,0,"[0.1791, 0.2505, 0.185, 0.1696, 0.1916, 0.144,...","[0.8209, 0.7495, 0.815, 0.8304, 0.8084, 0.856,...",0.076923,0.635933
1,"[0.4922, 0.4772, 0.4759, 0.523, 0.4546, 0.4869...",0,3,"[0.4043, 0.5356, 0.4414, 0.3749, 0.5262, 0.485...","[0.5957, 0.4644, 0.5586, 0.6251, 0.4738, 0.514...",NaN,NaN
2,"[0.4932, 0.4753, 0.4488, 0.471, 0.4637, 0.5183...",0,5,"[0.309, 0.2722, 0.1819, 0.2133, 0.3147, 0.2574...","[0.691, 0.7278, 0.8181, 0.7867, 0.6853, 0.7426...",NaN,NaN
3,"[0.4566, 0.4401, 0.4379, 0.5099, 0.4933, 0.473...",1,8,"[0.5852, 0.5284, 0.3385, 0.3547, 0.4242, 0.676...","[0.4148, 0.4716, 0.6615, 0.6453, 0.5758, 0.323...",NaN,NaN
4,"[0.4709, 0.4601, 0.4392, 0.4873, 0.4568, 0.488...",1,5,"[0.1871, 0.1223, 0.1232, 0.0604, 0.138, 0.1706...","[0.8129, 0.8777, 0.8768, 0.9396, 0.862, 0.8294...",NaN,NaN
5,"[0.4835, 0.4629, 0.4307, 0.4826, 0.4811, 0.487...",2,25,"[0.1896, 0.1677, 0.1654, 0.4521, 0.2145, 0.322...","[0.8104, 0.8323, 0.8346, 0.5479, 0.7855, 0.677...",NaN,NaN
6,"[0.564, 0.3654, 0.4858, 0.5078, 0.5714, 0.4903...",3,4,"[0.7427, 0.6975, 0.6756, 0.7571, 0.6212, 0.731...","[0.2573, 0.3025, 0.3244, 0.2429, 0.3788, 0.268...",NaN,NaN
7,"[0.5211, 0.439, 0.4182, 0.4895, 0.5126, 0.4956...",3,19,"[0.4619, 0.6138, 0.5258, 0.6339, 0.4549, 0.517...","[0.5381, 0.3862, 0.4742, 0.3661, 0.5451, 0.482...",NaN,NaN
8,"[0.4887, 0.4711, 0.4363, 0.4725, 0.49, 0.5042,...",3,19,"[0.3805, 0.3431, 0.3973, 0.5067, 0.3287, 0.387...","[0.6195, 0.6569, 0.6027, 0.4933, 0.6713, 0.612...",NaN,NaN
9,"[0.4387, 0.4702, 0.4792, 0.4366, 0.4952, 0.464...",4,20,"[0.8979, 0.8387, 0.8884, 0.9371, 0.8851, 0.945...","[0.1021, 0.1613, 0.1116, 0.0629, 0.1149, 0.054...",NaN,NaN


41 : Training: loss:  0.6499051
42 : Training: loss:  0.64526373
43 : Training: loss:  0.6313186
44 : Training: loss:  0.6166663
45 : Training: loss:  0.63606256
46 : Training: loss:  0.6220453
47 : Training: loss:  0.6131759
48 : Training: loss:  0.6202844
49 : Training: loss:  0.6073287
50 : Training: loss:  0.60567755
51 : Training: loss:  0.61508983
52 : Training: loss:  0.60600305
53 : Training: loss:  0.6042247
54 : Training: loss:  0.5964525
55 : Training: loss:  0.6206141
56 : Training: loss:  0.5856006
57 : Training: loss:  0.5996536
58 : Training: loss:  0.5899961
59 : Training: loss:  0.6082875
60 : Training: loss:  0.5749101
Validation: Loss:  0.5809698  Accuracy:  0.09615385


...

,Class scores,Correct labels,Predicted labels,Trust to textual modality,Trust to visual modality,Accuracy,Loss
0,"[0.818, 0.0504, 0.0038, 0.0018, 0.0036, 0.0202...",0,0,"[0.6064, 0.7208, 0.5556, 0.1658, 0.4359, 0.218...","[0.3936, 0.2792, 0.4444, 0.8342, 0.5641, 0.781...",0.980769,0.031869
1,"[0.9072, 0.1477, 0.0002, 0.0011, 0.0325, 0.018...",0,0,"[0.8217, 0.8188, 0.6852, 0.2731, 0.9132, 0.539...","[0.1783, 0.1812, 0.3148, 0.7269, 0.0868, 0.460...",NaN,NaN
2,"[0.9221, 0.0406, 0.0008, 0.0024, 0.0215, 0.136...",0,0,"[0.9241, 0.7559, 0.7418, 0.1855, 0.5849, 0.406...","[0.0759, 0.2441, 0.2582, 0.8145, 0.4151, 0.593...",NaN,NaN
3,"[0.0062, 0.289, 0.0002, 0.0003, 0.039, 0.0409,...",1,1,"[0.8718, 0.2283, 0.4776, 0.132, 0.9042, 0.833,...","[0.1282, 0.7717, 0.5224, 0.868, 0.0958, 0.167,...",NaN,NaN
4,"[0.0359, 0.1804, 0.0011, 1e-04, 0.0026, 0.0154...",1,1,"[0.5284, 0.0648, 0.2815, 0.0488, 0.5825, 0.382...","[0.4716, 0.9352, 0.7185, 0.9512, 0.4175, 0.617...",NaN,NaN
5,"[0.0007, 0.0029, 0.8904, 0.003, 0.0003, 0.0069...",2,2,"[0.3276, 0.6352, 0.3304, 0.8892, 0.5887, 0.872...","[0.6724, 0.3648, 0.6696, 0.1108, 0.4113, 0.127...",NaN,NaN
6,"[0.0054, 0.0008, 0.0003, 0.9576, 0.0012, 0.000...",3,3,"[0.9375, 0.6845, 0.9783, 0.9576, 0.4689, 0.877...","[0.0625, 0.3155, 0.0217, 0.0424, 0.5311, 0.122...",NaN,NaN
7,"[0.0062, 0.0011, 0.0038, 0.7723, 0.0022, 0.000...",3,3,"[0.5842, 0.7276, 0.8568, 0.9255, 0.3708, 0.873...","[0.4158, 0.2724, 0.1432, 0.0745, 0.6292, 0.126...",NaN,NaN
8,"[0.0031, 0.0006, 0.0066, 0.7678, 0.0075, 0.000...",3,3,"[0.533, 0.7733, 0.8596, 0.9601, 0.1792, 0.8716...","[0.467, 0.2267, 0.1404, 0.0399, 0.8208, 0.1284...",NaN,NaN
9,"[0.0019, 0.0055, 1e-04, 0.0012, 0.9193, 0.0863...",4,4,"[0.8847, 0.5761, 0.7432, 0.8184, 0.9013, 0.978...","[0.1153, 0.4239, 0.2568, 0.1816, 0.0987, 0.022...",NaN,NaN


3461 : Training: loss:  0.020307643
3462 : Training: loss:  0.022390468
3463 : Training: loss:  0.018955223
3464 : Training: loss:  0.014792187
3465 : Training: loss:  0.021680985
3466 : Training: loss:  0.018736405
3467 : Training: loss:  0.013619108
3468 : Training: loss:  0.017074976
3469 : Training: loss:  0.014444548
3470 : Training: loss:  0.009161453
3471 : Training: loss:  0.017166277
3472 : Training: loss:  0.01954225
3473 : Training: loss:  0.023267325
3474 : Training: loss:  0.012507081
3475 : Training: loss:  0.008827921
3476 : Training: loss:  0.014700625
3477 : Training: loss:  0.019193174
3478 : Training: loss:  0.032663804
3479 : Training: loss:  0.019790886
3480 : Training: loss:  0.018805025
Validation: Loss:  0.031561688  Accuracy:  0.96153843
3481 : Training: loss:  0.012342836
3482 : Training: loss:  0.010129641
3483 : Training: loss:  0.015403736
3484 : Training: loss:  0.024546945
3485 : Training: loss:  0.02135521
3486 : Training: loss:  0.018609276
3487 : Train

3693 : Training: loss:  0.022997508
3694 : Training: loss:  0.021552801
3695 : Training: loss:  0.02112647
3696 : Training: loss:  0.02804091
3697 : Training: loss:  0.013802712
3698 : Training: loss:  0.011038356
3699 : Training: loss:  0.014008161
3700 : Training: loss:  0.008606775
Validation: Loss:  0.028405912  Accuracy:  0.96153843
3701 : Training: loss:  0.011141682
3702 : Training: loss:  0.015946269
3703 : Training: loss:  0.01590927
3704 : Training: loss:  0.041300774
3705 : Training: loss:  0.018335223
3706 : Training: loss:  0.017181111
3707 : Training: loss:  0.016413592
3708 : Training: loss:  0.014019536
3709 : Training: loss:  0.014932415
3710 : Training: loss:  0.011560519
3711 : Training: loss:  0.014093186
3712 : Training: loss:  0.011166283
3713 : Training: loss:  0.015213975
3714 : Training: loss:  0.013322578
3715 : Training: loss:  0.02169269
3716 : Training: loss:  0.013770266
3717 : Training: loss:  0.0077566453
3718 : Training: loss:  0.009261774
3719 : Traini

3922 : Training: loss:  0.015751122
3923 : Training: loss:  0.019591916
3924 : Training: loss:  0.012757687
3925 : Training: loss:  0.007014795
3926 : Training: loss:  0.0094243605
3927 : Training: loss:  0.009530912
3928 : Training: loss:  0.009640073
3929 : Training: loss:  0.009716369
3930 : Training: loss:  0.00850913
3931 : Training: loss:  0.014700668
3932 : Training: loss:  0.011129497
3933 : Training: loss:  0.014284315
3934 : Training: loss:  0.00926385
3935 : Training: loss:  0.007347434
3936 : Training: loss:  0.02299747
3937 : Training: loss:  0.009690184
3938 : Training: loss:  0.019237893
3939 : Training: loss:  0.018350717
3940 : Training: loss:  0.011460886
Validation: Loss:  0.025981229  Accuracy:  0.96153843
3941 : Training: loss:  0.009533787
3942 : Training: loss:  0.010042326
3943 : Training: loss:  0.0084808525
3944 : Training: loss:  0.01274008
3945 : Training: loss:  0.011487699
3946 : Training: loss:  0.007111778
3947 : Training: loss:  0.014999133
3948 : Train

,Class scores,Correct labels,Predicted labels,Trust to textual modality,Trust to visual modality,Accuracy,Loss
0,"[0.8686, 0.0266, 0.0013, 0.0009, 0.0021, 0.007...",0,0,"[0.634, 0.7621, 0.6152, 0.1771, 0.4303, 0.2025...","[0.366, 0.2379, 0.3848, 0.8229, 0.5697, 0.7975...",1.0,0.024199
1,"[0.9469, 0.1054, 1e-04, 0.0006, 0.0324, 0.0063...",0,0,"[0.8452, 0.846, 0.7322, 0.2813, 0.9232, 0.5142...","[0.1548, 0.154, 0.2678, 0.7187, 0.0768, 0.4858...",NaN,NaN
2,"[0.9636, 0.0242, 0.0003, 0.0011, 0.0208, 0.071...",0,0,"[0.9402, 0.7979, 0.8021, 0.1867, 0.5969, 0.389...","[0.0598, 0.2021, 0.1979, 0.8133, 0.4031, 0.610...",NaN,NaN
3,"[0.0031, 0.2568, 1e-04, 1e-04, 0.0743, 0.0222,...",1,1,"[0.9014, 0.2079, 0.509, 0.121, 0.9272, 0.8351,...","[0.0986, 0.7921, 0.491, 0.879, 0.0728, 0.1649,...",NaN,NaN
4,"[0.0218, 0.1588, 0.0003, 0.0, 0.0021, 0.0073, ...",1,1,"[0.5503, 0.0615, 0.308, 0.0498, 0.5982, 0.3744...","[0.4497, 0.9385, 0.692, 0.9502, 0.4018, 0.6256...",NaN,NaN
5,"[0.0002, 0.001, 0.9415, 0.0014, 0.0002, 0.0046...",2,2,"[0.3557, 0.6675, 0.3895, 0.922, 0.6284, 0.8806...","[0.6443, 0.3325, 0.6105, 0.078, 0.3716, 0.1194...",NaN,NaN
6,"[0.0033, 0.0003, 1e-04, 0.9731, 0.0009, 1e-04,...",3,3,"[0.9388, 0.6634, 0.9795, 0.9598, 0.4508, 0.875...","[0.0612, 0.3366, 0.0205, 0.0402, 0.5492, 0.124...",NaN,NaN
7,"[0.0032, 0.0005, 0.0016, 0.8243, 0.0021, 1e-04...",3,3,"[0.6041, 0.7327, 0.8562, 0.9332, 0.392, 0.8788...","[0.3959, 0.2673, 0.1438, 0.0668, 0.608, 0.1212...",NaN,NaN
8,"[0.0017, 0.0002, 0.0033, 0.8352, 0.0088, 0.000...",3,3,"[0.5474, 0.7864, 0.8708, 0.9679, 0.1884, 0.885...","[0.4526, 0.2136, 0.1292, 0.0321, 0.8116, 0.114...",NaN,NaN
9,"[0.0008, 0.002, 0.0, 0.0005, 0.9735, 0.0302, 1...",4,4,"[0.8998, 0.5678, 0.7177, 0.808, 0.8962, 0.9777...","[0.1002, 0.4322, 0.2823, 0.192, 0.1038, 0.0223...",NaN,NaN
